In [1]:
from src.classes.network import RandomNetwork, ScaleFreeNetwork
from src.classes.node import Node
from src.experimentation import generate_networks, read_and_load_networks, multiple_correlations_par
from src.viusalization import plot_cascade_animation, test_significance, calculate_average_per_gamma, plot_cascades_gamma #,plot_cascade_dist_average
from collections import defaultdict
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
from collections import defaultdict
import os, sys
import numpy as np



#### Global Values

In [2]:
correlations = np.linspace(-1, 1, 11)
correlations = np.round(correlations, 1)
initial_seeds = np.linspace(13, 1600, 11)
num_runs = 30
num_nodes = 200
update_fraction = 0.1
average_degree = 8
starting_distribution = 0.5     # L / R ratio (niet per se nodig maar kan misschien leuk zijn om te varieern)
p = average_degree/(num_nodes-1) 
updates = 300000
m = 2

# vary between random and scale_free
# what_net = "random"
what_net = "scale_free"

### Hoi dit is ff uitleg voor jullie voor de nieuwe classes:

Toevoegen van ScaleFreeNetwork

In [3]:
# # Gebruik plot=True om begin plot van distributie te zien
# network = ScaleFreeNetwork(m=m, plot=True)

# for round in range(10000):
#     network.update_round()

# # Gebruik deze functie om de distributie te plotten op het einde
# network.verify_scale_free_distribution(plot=True)

Toevoegen van RandomNetwork

In [4]:
# IPV Network() aan te roepen doe je nu RandomNetwork() of ScaleFreeNetwork()
# # Je kunt rustig dezelfde argumenten meegeven als bij Network() zoals p=0.1, k=8. Als dit wordt leeggelaaten worden standaard waarden gebruikt.

# random_network = RandomNetwork()
# for round in range(10000):
#     random_network.update_round()

#### Developing and saving network

In [5]:
# dummy values
# num_runs = 10
# updates=1000
# scale-free
# generate_networks(correlations, initial_seeds, num_nodes=num_nodes, iterations=updates, how_many=num_runs, update_fraction=update_fraction, starting_distribution=starting_distribution, p=p, network_sort=what_net, m=m)

 #### Reading in and generating Network

In [ ]:
# # dummy values
# num_runs = 5
# updates=1000
test=False

all_networks = read_and_load_networks(num_runs, num_nodes, update_fraction, average_degree, starting_distribution, correlations, whichtype=what_net)
used_seed = int(initial_seeds[0])
if what_net == "scale_free":
    test_network = ScaleFreeNetwork(num_nodes=num_nodes, m=m, mean=0, correlation=-1.0, update_fraction=update_fraction, starting_distribution=starting_distribution, seed=used_seed)    ### !!!!!!!!!!!!!!
else: 
    test_network = RandomNetwork(num_nodes=num_nodes, mean=0, correlation=-1.0, update_fraction=update_fraction, starting_distribution=starting_distribution, seed=used_seed, p=p)
number_of_alterations = 0

if test:

    assert set([(conn[0].ID, conn[1].ID) for conn in all_networks[(-1.0, 0)][0].connections]) == set([(conn[0].ID, conn[1].ID) for conn in test_network.connections]), "The networks that are generated should be the same at the start"

    for _ in range(updates):
        test_network.update_round()
        number_of_alterations += test_network.alterations
        test_network.clean_network()    
        
    assert set([(conn[0].ID, conn[1].ID) for conn in all_networks[(-1.0, 0)][1].connections]) == set([(conn[0].ID, conn[1].ID) for conn in test_network.connections]), "The networks that are generated should be the same at the end"



#### Analyzing and plotting the Cascade distributions

## Calculating the cascades (parallelized implementation)
### Reading in network and saving everything in datastructures. structured by size and correlation value. 


In [ ]:
cascades_before = defaultdict(lambda: defaultdict(list))
cascades_after = defaultdict(lambda: defaultdict(list))
cascades_before_averaged = defaultdict(lambda: defaultdict(list))
cascades_after_averaged = defaultdict(lambda: defaultdict(list))
save=True
sizes = defaultdict()
sizes_averaged = defaultdict()


for corr in correlations: 
    print(f"starting experimentation for correlation: {corr}")
    print("-----------------------------------------------")

    (before_after, before_after_averaged, largest_sizes) = multiple_correlations_par(corr, num_runs, num_nodes, update_fraction, average_degree, starting_distribution,what_net)
    (collection_of_all_before, collection_of_all_after) = before_after
    (coll_of_all_before_averaged, coll_of_all_after_averaged) = before_after_averaged
    (largest_size_of_all, largest_size_of_all_averaged) = largest_sizes
    
    sizes[corr] = largest_size_of_all
    cascades_before[corr] = collection_of_all_before
    cascades_after[corr] = collection_of_all_after

    sizes_averaged[corr] = largest_size_of_all_averaged
    cascades_before_averaged[corr] = coll_of_all_before_averaged
    cascades_after_averaged[corr] = coll_of_all_after_averaged


#### Annimation of cascade size distribution with average polarization. (average cascade size per sampled node is calculated)

In [ ]:
plot_cascade_animation(cascades_before_averaged, cascades_after_averaged, list(reversed(correlations)), sizes_averaged, num_runs, what_net, save=True, averaged=True)

##### Summarizing all cascade info in one plot 

In [ ]:
# for key,value in cascades_before_averaged.items():
#     plot_cascade_dist_average(value, "before", largest_size_of_all_averaged, num_runs, save, key)

# for key,value in cascades_after_averaged.items():
#     plot_cascade_dist_average(value, "after", largest_size_of_all_averaged, num_runs, save, key)


plot_cascades_gamma((cascades_before_averaged, cascades_after_averaged), num_runs, what_net)

#### Calculating significance between cascade sizes and polarizations between networks before and after updating, for different gamma values

In [ ]:

# Run the test on your data
values_bef, values_af, variance_bef, variance_af = calculate_average_per_gamma((cascades_before_averaged, cascades_after_averaged), num_runs)
results = test_significance(values_bef, values_af, variance_bef, variance_af, num_runs)

# Print results

print(f"Statistical significance for {what_net} network type (cascade experiments)")
for gamma, res in results.items():
    print(f"Gamma = {gamma}:")
    print(f"  Size: t = {res['t_size']:.3f}, p = {res['p_size']:.3g}")
    print(f"  Polarization: t = {res['t_pol']:.3f}, p = {res['p_pol']:.3g}")
    print("-" * 40)
